In [1]:
import math
import os

import matplotlib.pyplot as plt
import numpy as np
import preparers
import sonaion_analysis as son

(1/6) Construct Paths
(2/6) Read Eye Tracker Data
(3/6) Transform Eye Tracker Data
(4/6) Normalize Eye Tracker Time
(5/6) Read PsychoPy Data
(6/6) Transform PsychoPy Data


In [3]:
from sonaion_analysis.utils.smallestenclosingcircle import make_circle


def mask_function():
    return lambda height, width, coordinate: son.utils.masks.create_circular_mask(height, width, coordinate, 30)


def interpolate(start_color, end_color, current, max_value):
    t = current / float(max_value)
    r = (1.0 - t) * start_color[0] + t * end_color[0]
    g = (1.0 - t) * start_color[1] + t * end_color[1]
    b = (1.0 - t) * start_color[2] + t * end_color[2]
    return (r, g, b)


def create_fixxation_map(eye_x, eye_y, fixxation_classifier):
    points_array = []
    currently_fixxation = False
    current_points = []

    for idx, classifier in enumerate(fixxation_classifier):
        if classifier == 1 and currently_fixxation == False:
            current_points = [(eye_x[idx], eye_y[idx])]
            currently_fixxation = True
        elif classifier == 1:
            current_points.append((eye_x[idx], eye_y[idx]))
        elif classifier == 0 and currently_fixxation == True:
            points_array.append((current_points.copy(), True))
            current_points = []
            currently_fixxation = False
            points_array.append(([(eye_x[idx], eye_y[idx])], False))
        else:
            points_array.append(([(eye_x[idx], eye_y[idx])], False))

    circles = [(make_circle(points), is_fixxation) for points, is_fixxation in points_array]
    circles = [((x, y), radius, is_fixxation) for ((x, y, radius), is_fixxation) in circles]

    return circles

In [28]:
dt = 0.008595
display_w = 1920
display_h = 1080

for i in [4, 5, 6, 8, 9, 10]:
    print(i)
    df_snippet, df_eyetracking = preparers.get_data(i)
    prefix = "./filteredData/" + str(i) + "/"
    os.makedirs(prefix, exist_ok=True)

    for idx, row in df_snippet.iterrows():
        try:
            image = "./images/" + row["Snippet"] + ".png"
            image_name = row["Snippet"]
            print("Calculate " + image_name)

            image = plt.imread(image)
            start = row["Start"]
            stop = row["Stop"]
            df_eye = df_eyetracking[(df_eyetracking["time"] >= start) & (df_eyetracking["time"] < stop)]

            left_color = (1.0, 0.0, 0.0)
            right_color = (0.0, 1.0, 0.0)

            #########################################################################
            # transform data to screen coordinates
            print("\t(1/7): Transforming Eytracking Data to Screen Coordinates")
            df_eye["l_display_x"] = df_eye["l_display_x"].apply(lambda x: x * display_w if (math.isnan(x) == False) else -1)
            df_eye["l_display_x"] = df_eye["l_display_x"].astype(int)

            df_eye["r_display_x"] = df_eye["r_display_x"].apply(lambda x: x * display_w if (math.isnan(x) == False) else -1)
            df_eye["r_display_x"] = df_eye["r_display_x"].astype(int)

            df_eye["l_display_y"] = df_eye["l_display_y"].apply(lambda x: x * display_h if (math.isnan(x) == False) else -1)
            df_eye["l_display_y"] = df_eye["l_display_y"].astype(int)

            df_eye["r_display_y"] = df_eye["r_display_y"].apply(lambda x: x * display_h if (math.isnan(x) == False) else -1)
            df_eye["r_display_y"] = df_eye["r_display_y"].astype(int)

            # reset the index for further processing
            df_eye = df_eye.reset_index()

            #########################################################################
            # preprocess eyetracking data
            print("\t(2/7): preprocess eyetracking data")
            replaced_r_x, replaced_r_y, replaced_r_diameter, replaced_r_valid = son.eyetracking.preprocessing_invalid.replace_with_prev_invalid(
                df_eye["r_display_x"], df_eye["r_display_y"], df_eye["r_pupil_diameter"], df_eye["r_valid"]
            )
            df_eye["r_display_x"] = replaced_r_x
            df_eye["r_display_y"] = replaced_r_y
            df_eye["r_pupil_diameter"] = replaced_r_diameter
            df_eye["r_valid"] = replaced_r_valid

            replaced_l_x, replaced_l_y, replaced_l_diameter, replaced_l_valid = son.eyetracking.preprocessing_invalid.replace_with_prev_invalid(
                df_eye["l_display_x"], df_eye["l_display_y"], df_eye["l_pupil_diameter"], df_eye["l_valid"]
            )

            df_eye["l_display_x"] = replaced_l_x
            df_eye["l_display_y"] = replaced_l_y
            df_eye["l_pupil_diameter"] = replaced_l_diameter
            df_eye["l_valid"] = replaced_l_valid

            for _i in range(50):
                denoised_r_x, denoised_r_y = son.eyetracking.preprocessing_denoise.denoise(df_eye["r_display_x"], df_eye["r_display_y"], son.eyetracking.preprocessing_denoise.parabola_5_kernel)

                denoised_l_x, denoised_l_y = son.eyetracking.preprocessing_denoise.denoise(df_eye["l_display_x"], df_eye["l_display_y"], son.eyetracking.preprocessing_denoise.parabola_5_kernel)

                df_eye["r_display_x"] = denoised_r_x
                df_eye["r_display_y"] = denoised_r_y

                df_eye["l_display_x"] = denoised_l_x
                df_eye["l_display_y"] = denoised_l_y

            #########################################################################
            # generate heat maps

            print("\t(3/7): Generate Heat Maps")
            dre = son.eyetracking.heatmap.create_heatmap(df_eye["r_display_x"], df_eye["r_display_y"], df_eye["r_valid"], 0.004, display_w, display_h, mask_function())
            dle = son.eyetracking.heatmap.create_heatmap(df_eye["l_display_x"], df_eye["l_display_y"], df_eye["l_valid"], 0.004, display_w, display_h, mask_function())

            # normalize heat
            dre = dre / dre.max()
            dle = dle / dle.max()

            dre = np.array([dre * right_color[0], dre * right_color[1], dre * right_color[2], dre])
            dle = np.array([dle * left_color[0], dle * left_color[1], dle * left_color[2], dle])

            data_right_heat = np.moveaxis(dre, 0, 2)
            data_left_heat = np.moveaxis(dle, 0, 2)

            #########################################################################
            # create sequence diagrams with vertical movement
            print("\t(4/7): Generate Sequence Diagrams for Y Movement")
            dre = son.eyetracking.sequence.create_sequence_diagram_y(df_eye["r_display_y"], df_eye["r_valid"], display_h, display_w, offset=0, step=0.1, should_skip=True)
            dle = son.eyetracking.sequence.create_sequence_diagram_y(df_eye["l_display_y"], df_eye["l_valid"], display_h, display_w, offset=0, step=0.1, should_skip=True)

            dre = np.array([dre * right_color[0], dre * right_color[1], dre * right_color[2], dre])
            dle = np.array([dle * left_color[0], dle * left_color[1], dle * left_color[2], dle])

            data_right_seq_y = np.moveaxis(dre, 0, 2)
            data_left_seq_y = np.moveaxis(dle, 0, 2)

            #########################################################################
            # create sequence diagrams with vertical movement
            print("\t(5/7): Generate Sequence Diagrams for X Movement")
            dre = son.eyetracking.sequence.create_sequence_diagram_x(df_eye["r_display_x"], df_eye["r_valid"], display_h, display_w, offset=0, step=0.1, should_skip=True)
            dle = son.eyetracking.sequence.create_sequence_diagram_x(df_eye["l_display_x"], df_eye["l_valid"], display_h, display_w, offset=0, step=0.1, should_skip=True)

            dre = np.array([dre * right_color[0], dre * right_color[1], dre * right_color[2], dre])
            dle = np.array([dle * left_color[0], dle * left_color[1], dle * left_color[2], dle])

            data_right_seq_x = np.moveaxis(dre, 0, 2)
            data_left_seq_x = np.moveaxis(dle, 0, 2)

            #########################################################################
            # fixxations and saccades
            print("\t(6/6): Fixxation Map Generation")
            treshhold = 2000.0
            d_time = 0.004

            right_saccades_color = (1.0, 0.0, 1.0)
            right_fixxation_color = (0.0, 1.0, 0.0)
            saccades_r = son.eyetracking.metrics.classify_saccades(df_eye["r_display_x"], df_eye["r_display_y"], d_time, treshhold)
            saccades_r[0] = saccades_r[1]
            fixxation_r = son.eyetracking.metrics.classify_fixxation(df_eye["r_display_x"], df_eye["r_display_y"], d_time, treshhold)
            fixxation_r[0] = fixxation_r[1]

            saccades_r = np.array(saccades_r)
            fixxation_r = np.array(fixxation_r)
            classificator_right = np.add(saccades_r, np.multiply(fixxation_r, 2))

            left_saccades_color = (0.0, 1.0, 1.0)
            left_fixxation_color = (1.0, 0.0, 0.0)
            saccades_l = son.eyetracking.metrics.classify_saccades(df_eye["l_display_x"], df_eye["l_display_y"], d_time, treshhold)
            saccades_l[0] = saccades_l[1]
            fixxation_l = son.eyetracking.metrics.classify_fixxation(df_eye["l_display_x"], df_eye["l_display_y"], d_time, treshhold)
            fixxation_l[0] = fixxation_l[1]

            saccades_l = np.array(saccades_l)
            fixxation_l = np.array(fixxation_l)
            classificator_left = np.add(saccades_l, np.multiply(fixxation_l, 2))

            fixxation_map_l = create_fixxation_map(df_eye["l_display_x"], df_eye["l_display_y"], fixxation_l)
            fixxation_map_r = create_fixxation_map(df_eye["r_display_x"], df_eye["r_display_y"], fixxation_r)

            #########################################################################
            # plot data
            print("\t(7/7): Generate Plot")

            # heatmaps
            fig, ax = plt.subplots()
            ax.set_axis_off()
            ax.imshow(data_left_heat, zorder=2, alpha=0.5)
            ax.imshow(image, zorder=1)
            fig.savefig(prefix + image_name + "_heatmap_left_eye.png", dpi=200)
            plt.close()

            fig, ax = plt.subplots()
            ax.set_axis_off()
            ax.imshow(data_right_heat, zorder=2, alpha=0.5)
            ax.imshow(image, zorder=1)
            fig.savefig(prefix + image_name + "_heatmap_right_eye.png", dpi=200)
            plt.close()

            fig, ax = plt.subplots()
            ax.set_axis_off()
            ax.imshow(data_left_heat, zorder=2, alpha=0.5)
            ax.imshow(data_right_heat, zorder=3, alpha=0.5)
            ax.imshow(image, zorder=1)
            fig.savefig(prefix + image_name + "_heatmap_eye.png", dpi=200)
            plt.close()

            # sequence y
            fig, ax = plt.subplots()
            ax.set_axis_off()
            ax.imshow(data_left_seq_y, zorder=2, alpha=0.5)
            ax.imshow(image, zorder=1)
            fig.savefig(prefix + image_name + "_x_sequence_diagram_left_eye.png", dpi=200)
            plt.close()

            fig, ax = plt.subplots()
            ax.set_axis_off()
            ax.imshow(data_right_seq_y, zorder=2, alpha=0.5)
            ax.imshow(image, zorder=1)
            fig.savefig(prefix + image_name + "_x_sequence_diagram_right_eye.png", dpi=200)
            plt.close()

            fig, ax = plt.subplots()
            ax.set_axis_off()
            ax.imshow(data_left_seq_y, zorder=2, alpha=0.5)
            ax.imshow(data_right_seq_y, zorder=3, alpha=0.5)
            ax.imshow(image, zorder=1)
            fig.savefig(prefix + image_name + "_x_sequence_diagram_eye.png", dpi=200)
            plt.close()

            # sequence x
            fig, ax = plt.subplots()
            ax.set_axis_off()
            ax.imshow(data_left_seq_x, zorder=2, alpha=0.5)
            ax.imshow(image, zorder=1)
            fig.savefig(prefix + image_name + "_y_sequence_diagram_left_eye.png", dpi=200)
            plt.close()

            fig, ax = plt.subplots()
            ax.set_axis_off()
            ax.imshow(data_right_seq_x, zorder=2, alpha=0.5)
            ax.imshow(image, zorder=1)
            fig.savefig(prefix + image_name + "_y_sequence_diagram_right_eye.png", dpi=200)
            plt.close()

            fig, ax = plt.subplots()
            ax.set_axis_off()
            ax.imshow(data_left_seq_x, zorder=2, alpha=0.5)
            ax.imshow(data_right_seq_x, zorder=3, alpha=0.5)
            ax.imshow(image, zorder=1)
            fig.savefig(prefix + image_name + "_y_sequence_diagram_eye.png", dpi=200)
            plt.close()

            # fixxation map
            fig, ax = plt.subplots()
            x_val = [float(int(x)) for ((x, _), _, _) in fixxation_map_l]
            y_val = [float(int(y)) for ((_, y), _, _) in fixxation_map_l]

            for i in range(len(x_val) - 2):
                ax.plot(x_val[i : i + 2], y_val[i : i + 2], "-", color=interpolate(left_color, (0.0, 0.0, 0.0), i, len(x_val)), zorder=2, alpha=0.4)

            filtered_x_val = [float(int(x)) for ((x, _), _, flag) in fixxation_map_l if flag]
            filtered_y_val = [float(int(y)) for ((_, y), _, flag) in fixxation_map_l if flag]

            ax.set_axis_off()
            ax.scatter(filtered_x_val, filtered_y_val, 10, c=[interpolate(left_color, (0.0, 0.0, 0.0), idx, len(x_val)) for idx, (_, _, flag) in enumerate(fixxation_map_l) if flag], zorder=3, alpha=0.4)
            ax.imshow(image, zorder=1)
            fig.savefig(prefix + image_name + "_trace_path_left_eye.png", dpi=200)
            plt.close()

            fig, ax = plt.subplots()
            x_val = [float(int(x)) for ((x, _), _, _) in fixxation_map_r]
            y_val = [float(int(y)) for ((_, y), _, _) in fixxation_map_r]

            for i in range(len(x_val) - 2):
                ax.plot(x_val[i : i + 2], y_val[i : i + 2], "-", color=interpolate(right_color, (0.0, 0.0, 0.0), i, len(x_val)), zorder=2, alpha=0.4)

            filtered_x_val = [float(int(x)) for ((x, _), _, flag) in fixxation_map_r if flag]
            filtered_y_val = [float(int(y)) for ((_, y), _, flag) in fixxation_map_r if flag]

            ax.set_axis_off()
            ax.scatter(filtered_x_val, filtered_y_val, 10, c=[interpolate(right_color, (0.0, 0.0, 0.0), idx, len(x_val)) for idx, (_, _, flag) in enumerate(fixxation_map_r) if flag], zorder=3, alpha=0.4)
            ax.imshow(image, zorder=1)
            fig.savefig(prefix + image_name + "_trace_path_right_eye.png", dpi=200)
            plt.close()

            # fixxation map over heatmap
            fig, ax = plt.subplots()
            x_val = [float(int(x)) for ((x, _), _, _) in fixxation_map_l]
            y_val = [float(int(y)) for ((_, y), _, _) in fixxation_map_l]

            for i in range(len(x_val) - 2):
                ax.plot(x_val[i : i + 2], y_val[i : i + 2], "-", color=interpolate(left_color, (0.0, 0.0, 0.0), i, len(x_val)), zorder=2, alpha=0.4)

            filtered_x_val = [float(int(x)) for ((x, _), _, flag) in fixxation_map_l if flag]
            filtered_y_val = [float(int(y)) for ((_, y), _, flag) in fixxation_map_l if flag]

            ax.set_axis_off()
            ax.scatter(filtered_x_val, filtered_y_val, 10, c=[interpolate(left_color, (0.0, 0.0, 0.0), idx, len(x_val)) for idx, (_, _, flag) in enumerate(fixxation_map_l) if flag], zorder=3, alpha=0.4)
            ax.imshow(data_left_heat, zorder=2, alpha=0.5)
            ax.imshow(image, zorder=1)
            fig.savefig(prefix + image_name + "_trace_path_heatmap_left_eye.png", dpi=200)
            plt.close()

            fig, ax = plt.subplots()
            x_val = [float(int(x)) for ((x, _), _, _) in fixxation_map_r]
            y_val = [float(int(y)) for ((_, y), _, _) in fixxation_map_r]

            for i in range(len(x_val) - 2):
                ax.plot(x_val[i : i + 2], y_val[i : i + 2], "-", color=interpolate(right_color, (0.0, 0.0, 0.0), i, len(x_val)), zorder=2, alpha=0.4)

            filtered_x_val = [float(int(x)) for ((x, _), _, flag) in fixxation_map_r if flag]
            filtered_y_val = [float(int(y)) for ((_, y), _, flag) in fixxation_map_r if flag]

            ax.set_axis_off()
            ax.scatter(filtered_x_val, filtered_y_val, 10, c=[interpolate(right_color, (0.0, 0.0, 0.0), idx, len(x_val)) for idx, (_, _, flag) in enumerate(fixxation_map_r) if flag], zorder=3, alpha=0.4)
            ax.imshow(data_right_heat, zorder=2, alpha=0.5)
            ax.imshow(image, zorder=1)
            fig.savefig(prefix + image_name + "_trace_path_heatmap_right_eye.png", dpi=200)
            plt.close()
        except:
            print("ERROR")

4
(1/6) Construct Paths
(2/6) Read Eye Tracker Data
(3/6) Transform Eye Tracker Data
(4/6) Normalize Eye Tracker Time
(5/6) Read PsychoPy Data
(6/6) Transform PsychoPy Data
Calculate Eyetracking
	(1/7): Transforming Eytracking Data to Screen Coordinates
	(2/7): preprocess eyetracking data
	(3/7): Generate Heat Maps
	(4/7): Generate Sequence Diagrams for Y Movement
	(5/7): Generate Sequence Diagrams for X Movement
	(6/6): Fixxation Map Generation
	(7/7): Generate Plot
Calculate Satz3
	(1/7): Transforming Eytracking Data to Screen Coordinates
	(2/7): preprocess eyetracking data
	(3/7): Generate Heat Maps
	(4/7): Generate Sequence Diagrams for Y Movement
	(5/7): Generate Sequence Diagrams for X Movement
	(6/6): Fixxation Map Generation
	(7/7): Generate Plot
Calculate Satz1
	(1/7): Transforming Eytracking Data to Screen Coordinates
	(2/7): preprocess eyetracking data
	(3/7): Generate Heat Maps
	(4/7): Generate Sequence Diagrams for Y Movement
	(5/7): Generate Sequence Diagrams for X Moveme

<ipython-input-28-1746b5f45e72>:82: RuntimeWarning: invalid value encountered in true_divide
  dre = dre / dre.max()
<ipython-input-28-1746b5f45e72>:83: RuntimeWarning: invalid value encountered in true_divide
  dle = dle / dle.max()


	(5/7): Generate Sequence Diagrams for X Movement
	(6/6): Fixxation Map Generation
ERROR
Calculate BrocantinySatz3
	(1/7): Transforming Eytracking Data to Screen Coordinates
	(2/7): preprocess eyetracking data
	(3/7): Generate Heat Maps
	(4/7): Generate Sequence Diagrams for Y Movement
	(5/7): Generate Sequence Diagrams for X Movement
	(6/6): Fixxation Map Generation
ERROR
Calculate BrocantinySatz2
	(1/7): Transforming Eytracking Data to Screen Coordinates
	(2/7): preprocess eyetracking data
	(3/7): Generate Heat Maps
	(4/7): Generate Sequence Diagrams for Y Movement
	(5/7): Generate Sequence Diagrams for X Movement
	(6/6): Fixxation Map Generation
ERROR
Calculate BrocantinySatz1
	(1/7): Transforming Eytracking Data to Screen Coordinates
	(2/7): preprocess eyetracking data
	(3/7): Generate Heat Maps
	(4/7): Generate Sequence Diagrams for Y Movement
	(5/7): Generate Sequence Diagrams for X Movement
	(6/6): Fixxation Map Generation
ERROR
Calculate programmErklaerung
	(1/7): Transforming 

<ipython-input-28-1746b5f45e72>:82: RuntimeWarning: invalid value encountered in true_divide
  dre = dre / dre.max()
<ipython-input-28-1746b5f45e72>:83: RuntimeWarning: invalid value encountered in true_divide
  dle = dle / dle.max()



Calculate BrocantinySatz3
	(1/7): Transforming Eytracking Data to Screen Coordinates
	(2/7): preprocess eyetracking data
	(3/7): Generate Heat Maps
	(4/7): Generate Sequence Diagrams for Y Movement
	(5/7): Generate Sequence Diagrams for X Movement
	(6/6): Fixxation Map Generation
	(7/7): Generate Plot
Calculate programmErklaerung
	(1/7): Transforming Eytracking Data to Screen Coordinates
	(2/7): preprocess eyetracking data
	(3/7): Generate Heat Maps
	(4/7): Generate Sequence Diagrams for Y Movement
	(5/7): Generate Sequence Diagrams for X Movement
	(6/6): Fixxation Map Generation
	(7/7): Generate Plot
Calculate ProgrammSatz3
	(1/7): Transforming Eytracking Data to Screen Coordinates
	(2/7): preprocess eyetracking data
	(3/7): Generate Heat Maps
	(4/7): Generate Sequence Diagrams for Y Movement
	(5/7): Generate Sequence Diagrams for X Movement
	(6/6): Fixxation Map Generation
	(7/7): Generate Plot
Calculate ProgrammSatz2
	(1/7): Transforming Eytracking Data to Screen Coordinates
	(2/7)

...

	(4/7): Generate Sequence Diagrams for Y Movement
	(5/7): Generate Sequence Diagrams for X Movement
	(6/6): Fixxation Map Generation
ERROR
Calculate ProgrammSatz2
	(1/7): Transforming Eytracking Data to Screen Coordinates
	(2/7): preprocess eyetracking data
	(3/7): Generate Heat Maps
	(4/7): Generate Sequence Diagrams for Y Movement
	(5/7): Generate Sequence Diagrams for X Movement
	(6/6): Fixxation Map Generation
ERROR
9
(1/6) Construct Paths
(2/6) Read Eye Tracker Data
(3/6) Transform Eye Tracker Data
(4/6) Normalize Eye Tracker Time
(5/6) Read PsychoPy Data
(6/6) Transform PsychoPy Data
Calculate Eyetracking
	(1/7): Transforming Eytracking Data to Screen Coordinates
	(2/7): preprocess eyetracking data
	(3/7): Generate Heat Maps
	(4/7): Generate Sequence Diagrams for Y Movement
	(5/7): Generate Sequence Diagrams for X Movement
	(6/6): Fixxation Map Generation
	(7/7): Generate Plot
Calculate Satz1
	(1/7): Transforming Eytracking Data to Screen Coordinates
	(2/7): preprocess eyetrackin

<ipython-input-28-1746b5f45e72>:82: RuntimeWarning: invalid value encountered in true_divide
  dre = dre / dre.max()
<ipython-input-28-1746b5f45e72>:83: RuntimeWarning: invalid value encountered in true_divide
  dle = dle / dle.max()


	(5/7): Generate Sequence Diagrams for X Movement
	(6/6): Fixxation Map Generation
ERROR
Calculate ProgrammSatz1
	(1/7): Transforming Eytracking Data to Screen Coordinates
	(2/7): preprocess eyetracking data
	(3/7): Generate Heat Maps
	(4/7): Generate Sequence Diagrams for Y Movement
	(5/7): Generate Sequence Diagrams for X Movement
	(6/6): Fixxation Map Generation
ERROR
10
(1/6) Construct Paths
(2/6) Read Eye Tracker Data
(3/6) Transform Eye Tracker Data
(4/6) Normalize Eye Tracker Time
(5/6) Read PsychoPy Data
(6/6) Transform PsychoPy Data
Calculate Eyetracking
	(1/7): Transforming Eytracking Data to Screen Coordinates
	(2/7): preprocess eyetracking data
	(3/7): Generate Heat Maps
	(4/7): Generate Sequence Diagrams for Y Movement
	(5/7): Generate Sequence Diagrams for X Movement
	(6/6): Fixxation Map Generation
	(7/7): Generate Plot
Calculate Satz3
	(1/7): Transforming Eytracking Data to Screen Coordinates
	(2/7): preprocess eyetracking data
ERROR
Calculate Satz2
	(1/7): Transforming

<ipython-input-28-1746b5f45e72>:82: RuntimeWarning: invalid value encountered in true_divide
  dre = dre / dre.max()
<ipython-input-28-1746b5f45e72>:83: RuntimeWarning: invalid value encountered in true_divide
  dle = dle / dle.max()


	(6/6): Fixxation Map Generation
	(7/7): Generate Plot
Calculate BrocantinySatz2
	(1/7): Transforming Eytracking Data to Screen Coordinates
	(2/7): preprocess eyetracking data
	(3/7): Generate Heat Maps
	(4/7): Generate Sequence Diagrams for Y Movement
	(5/7): Generate Sequence Diagrams for X Movement
	(6/6): Fixxation Map Generation
	(7/7): Generate Plot
Calculate BrocantinySatz1
	(1/7): Transforming Eytracking Data to Screen Coordinates
	(2/7): preprocess eyetracking data
	(3/7): Generate Heat Maps
	(4/7): Generate Sequence Diagrams for Y Movement
	(5/7): Generate Sequence Diagrams for X Movement
	(6/6): Fixxation Map Generation
	(7/7): Generate Plot
Calculate programmErklaerung
	(1/7): Transforming Eytracking Data to Screen Coordinates
	(2/7): preprocess eyetracking data
	(3/7): Generate Heat Maps
	(4/7): Generate Sequence Diagrams for Y Movement
	(5/7): Generate Sequence Diagrams for X Movement
	(6/6): Fixxation Map Generation
	(7/7): Generate Plot
Calculate ProgrammSatz2
	(1/7): T